# COVID-19 Data Analysis
Robust loader + quick checks and a first plot.

In [ ]:
# Robust data loader (works in Colab and locally)
import pandas as pd

PRIMARY = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
MIRROR  = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

def load_owid_covid():
    for url in (MIRROR, PRIMARY):
        try:
            print(f"Trying: {url}")
            df = pd.read_csv(url)
            print("Loaded from:", url)
            return df
        except Exception as e:
            print("Failed:", e)
    try:
        from google.colab import files
        print("Upload 'owid-covid-data.csv' from your PC…")
        uploaded = files.upload()
        fname = next(iter(uploaded))
        return pd.read_csv(fname)
    except Exception as e:
        raise RuntimeError("Could not load data from web or upload.") from e

df = load_owid_covid()
df.head()

In [ ]:
# Basic cleaning / column selection
keep = ["iso_code","continent","location","date",
        "total_cases","new_cases","total_deaths","new_deaths",
        "people_vaccinated","people_fully_vaccinated","total_tests","population"]
df = df[keep]
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.sort_values(['location','date'], inplace=True)
df.info()

In [ ]:
# Example 1: Global daily new cases
global_daily = (df.groupby('date')['new_cases']
                  .sum(min_count=1)
                  .fillna(0))

global_daily.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
global_daily.plot()
plt.title("Global Daily New COVID-19 Cases")
plt.xlabel("Date")
plt.ylabel("New Cases")
plt.tight_layout()
plt.savefig("covid_trends.png", dpi=150, bbox_inches="tight")
plt.savefig("covid_trends.svg", bbox_inches="tight")
plt.show()

In [ ]:
# Example 2: Top 10 countries by total cases
latest = df.dropna(subset=['total_cases']).sort_values(['location','date']).groupby('location').tail(1)
top10 = latest.sort_values('total_cases', ascending=False).head(10)[['location','total_cases']]
top10

In [ ]:
plt.figure(figsize=(10,5))
top10.set_index('location')['total_cases'].plot(kind='bar')
plt.title("Top 10 Countries by Total Cases (latest)")
plt.xlabel("Country")
plt.ylabel("Total Cases")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig("covid_top10_total_cases.png", dpi=150, bbox_inches="tight")
plt.savefig("covid_top10_total_cases.svg", bbox_inches="tight")
plt.show()